# KIKA Quickstart Guide

This notebook demonstrates the core functionality of KIKA, a Python package for working with MCNP (Monte Carlo N-Particle) files and data.

KIKA allows you to:
- Parse and manipulate MCNP input files
- Read and analyze MCTAL output files
- Compute sensitivity data
- Work with SDF (Sensitivity Data Format) files

Let's start by importing the package and setting up our environment:

In [1]:
import kika
from pathlib import Path
repo_root = Path.cwd().resolve().parent

## Working with MCNP Input Files

MCNP input files define the problem geometry, materials, source terms, and desired tallies. KIKA provides tools to parse, some of this information.

First, let's load an example input file:

In [2]:
inputfile = repo_root / 'examples' / 'data' / 'inputfile_example_1.i'

In [3]:
# Parse an input file
input_data = kika.read_mcnp(inputfile)

In [4]:
# Print a summary of the input file - shows key information about the model
input_data

                      MCNP Input Data                       

------------------------------------------------------------
                         MATERIALS                          
------------------------------------------------------------
Number of materials:      4
Material IDs:             100000, 200100, 300000, 300001
Atomic fraction mats:     4

Use .materials to access material data.

------------------------------------------------------------
                       PERTURBATIONS                        
------------------------------------------------------------
Number of perturbations:  1760
Perturbation IDs:         1-1760
Reactions available:      1, 2, 4, 51, 52, 102, 103, 107
Methods available:        -3, -2, 1, 2, 3
Perturbed materials:      300001
Energy range:             1.00e-11 - 2.00e+01 MeV
Energy bins:              44
Energy structure:         SCALE44

Use .perturbation to access perturbation data.

### Accessing Input Components

The parsed input file provides structured access to different components such as materials and perturbation cards:

In [5]:
# Access materials or PERT cards (if available)
materials = input_data.materials
perturbations = input_data.perturbation

In [6]:
# Print a summary of materials - shows material IDs and compositions
materials

                 MCNP Materials Collection                  

Total Materials: 4

------------------------------------------------------------
    ID    |   Nuclides    |     Type      |    Libraries     
------------------------------------------------------------
  100000  |       2       |    Atomic     |       06c        
  200100  |       4       |    Atomic     |       06c        
  300000  |      36       |    Atomic     |       06c        
  300001  |      36       |    Atomic     |       06c        
------------------------------------------------------------

Available methods:
- .to_weight_fractions() - Convert all materials to weight fractions
- .to_atomic_fractions() - Convert all materials to atomic fractions

Examples of accessing data:
- .mat[material_id] - Access a specific material

In [7]:
# Print materials in MCNP format - as they would appear in an input file
print(materials)

   MCNP Materials Collection (4 materials)

            MCNP Material (ID: 100000)            

Properties: nlib=06c
Total Nuclides: 2

--------------------------------------------------
   ZAID   | Element  |   Fraction    |  Libraries  
--------------------------------------------------
   7014   |    N     |  7.8848e-01   |      -      
   8016   |    O     |  2.1152e-01   |      -      
--------------------------------------------------

Available methods:
- .to_weight_fraction() - Convert material to weight fractions
- .to_atomic_fraction() - Convert material to atomic fractions
- .convert_natural_elements() - Convert natural elements into isotopic composition
- .copy(new_id) - Create a copy with a new material ID

Examples of accessing data:
- .nuclide[zaid] - Access a specific nuclide
- print(material) - Print material in MCNP format

            MCNP Material (ID: 200100)            

Properties: nlib=06c
Total Nuclides: 4

--------------------------------------------------
   

In [8]:
# Print a summary of perturbations - shows any PERT cards in the input
perturbations

                   MCNP Perturbation Data                   

Number of perturbations:  1760
Perturbation numbers:     1-1760
Particle types:           n
Reactions available:      1, 2, 4, 51, 52, 102, 103, 107
Methods available:        -3, -2, 1, 2, 3
Energy range:             1.00e-11 - 2.00e+01 MeV
Number of energy bins:    44
Energy structure:         SCALE44


Examples of accessing data:
- .pert[perturbation_number] - Access a specific perturbation

## Working with MCTAL Files

MCTAL files contain tally results from MCNP simulations. KIKA allows you to extract and analyze this data.

Let's load and examine an example MCTAL file:

In [9]:
mctalfile = repo_root / 'examples' / 'data' / 'mctalfile_example_1.m'

In [10]:
# Parse a mctal file
mctal = kika.read_mctal(mctalfile)

In [11]:
# Print a summary of the mctal file - shows available tallies and problem info
mctal

                          MCTAL File Summary                          

               Code: mcnp6 6
         Problem ID: 05/24/24 17:18:48
                NPS: 1.00e+07
              Title: attenuation Sphere for PWR

Tally Information:
      Total Tallies: 1

Tally Type Distribution:
----------------------------------------------------------------------
   Type   |         Description          |         Tally IDs          
----------------------------------------------------------------------
    F4    |          Cell flux           |4                           
----------------------------------------------------------------------


Perturbation Data Summary:
----------------------------------------------------------------------
   Tally ID    |      Perturbations      |      Perturbation IDs      
----------------------------------------------------------------------
       4       |          1760           |1-1760                      
---------------------------------------------

### Accessing Tallies

You can extract specific tallies from the MCTAL file to examine the results in detail:

In [12]:
# Access tallies by their ID number
tally = mctal.tally[4]

In [13]:
# Print a summary of the tally - shows type, binning structure, and result overview
tally

                                    MCNP Tally 4                                     
                                   shell Detector                                    

Dimensions:
-------------------------------------------------------------------------------------
Dimension            Size           
-------------------------------------------------------------------------------------
Cell/Surface         1              
Energy               3              
-------------------------------------------------------------------------------------
Total Results: 3


Additional Information:
• Integral energy data available
• 1760 perturbations
• 20 TFC data points


Available Methods:
-------------------------------------------------------------------------------------
Method                            Description                                      
-------------------------------------------------------------------------------------
.to_dataframe()                   Get results as pa

## Working with Sensitivities

KIKA can compute sensitivity data, which shows how results change with respect to nuclear data parameters.

Here we'll calculate sensitivity for a specific nuclide:

In [14]:
inputfile = repo_root / 'examples' / 'data' / 'inputfile_example_1.i'
mctalfile = repo_root / 'examples' / 'data' / 'mctalfile_example_1.m'

In [15]:
# Compute sensitivity data for Fe-56 (ZAID 26056) for tally 4
sensdata = kika.compute_sensitivity(
    inputfile = inputfile,
    mctalfile = mctalfile,
    tally = 4,
    zaid = 26056,
    label = 'Sensitivity-26056'
)

In [16]:
# Print a summary of the sensitivity data
sensdata

                             Sensitivity Data for Fe-56                              

Label:                           Sensitivity-26056
Tally ID:                        4
Tally Name:                      shell Detector
Nuclide (ZAID):                  Fe-56 (26056)
Number of detector energy bins:  3
Number of perturbation bins:     44
Energy structure:                SCALE44
Reactions available:             1, 2, 4, 51, 52, 102, 103, 107
Second Order data available:     True

Energy group ranges:
  - 0.00e+00_1.00e-01
  - 1.00e-01_1.00e+00
  - 1.00e+00_3.00e+00
  - integral


Available Methods:
-------------------------------------------------------------------------------------
Method                                 Description                                 
-------------------------------------------------------------------------------------
.plot_sensitivity(...)                 Plot sensitivity profiles                   
.to_dataframe()                        Get full data as 

## Working with SDF Files

Sensitivity Data Format (SDF) files store sensitivity profiles for multiple nuclides.
KIKA can create SDF data from multiple sensitivity calculations and write it to standard format files.

First, let's load input and output files with perturbations for different nuclides:

In [17]:
# Read input/output files each one with the perturbation of a different nuclide
inputfile1 = repo_root / 'examples' / 'data' / 'inputfile_example_1.i'
inputfile2 = repo_root / 'examples' / 'data' / 'inputfile_example_2.i'
mctalfile1 = repo_root / 'examples' / 'data' / 'mctalfile_example_1.m'
mctalfile2 = repo_root / 'examples' / 'data' / 'mctalfile_example_2.m'

Now calculate sensitivity data for each case:

In [18]:
# Calculate sensitivity for iron-56 and hydrogen-1
sensdata1 = kika.compute_sensitivity(inputfile1, mctalfile1, 4, 26056, 'Sensitivity Fe56')
sensdata2 = kika.compute_sensitivity(inputfile2, mctalfile2, 4, 1001, 'Sensitivity H1')

In [19]:
# Create a list with the sensitivity data
sensdata_list = [sensdata1, sensdata2]

In [20]:
# Print detector energy bins - these define the energy structure for our sensitivity profiles
sensdata1.energies

['0.00e+00_1.00e-01', '1.00e-01_1.00e+00', '1.00e+00_3.00e+00', 'integral']

### Creating and Writing SDF Data

We can create an SDF data object for a specific energy range and write it to a file:

In [21]:
# Create an sdf data object with the list of sensitivity data for an energy bin
sdf_data = kika.create_sdf_data(sensdata_list, energy='1.00e+00_3.00e+00', title='Example SDF Data')

Skipping H-1 inelastic (MT=4): All sensitivity coefficients are zero
Skipping H-1 inelastic-1st (MT=51): All sensitivity coefficients are zero
Skipping H-1 inelastic-2nd (MT=52): All sensitivity coefficients are zero
Skipping H-1 n,p (MT=103): All sensitivity coefficients are zero
Skipping H-1 n,alpha (MT=107): All sensitivity coefficients are zero


In [22]:
# Print a summary of the sdf data
sdf_data

                      SDF Data: Example SDF Data                      
                   Energy range: 1.00e+00_3.00e+00                    

Response value:           1.066520e-09 ± 1.73% (rel)
Energy groups:            44
Energy structure:         SCALE44
Sensitivity profiles:     11
Unique nuclides:          2

Energy grid (preview):
    1.000000e-11, 3.000000e-09, 7.500000e-09, ... , 6.434000e+00, 8.187300e+00, 2.000000e+01

Nuclides and reactions (with access indices):

  Fe-56:
    total (MT=1)                   access with .data[0]
    elastic (MT=2)                 access with .data[1]
    inelastic (MT=4)               access with .data[2]
    inelastic-1st (MT=51)          access with .data[3]
    inelastic-2nd (MT=52)          access with .data[4]
    n,gamma (MT=102)               access with .data[5]
    n,p (MT=103)                   access with .data[6]
    n,alpha (MT=107)               access with .data[7]

  H-1:
    total (MT=1)                   access with .data[8

In [23]:
# Write the sdf data to a sdf format file
sdf_data.write_file()

SDF file saved successfully: /home/MONLEON-JUAN/KIKA/examples/Example_SDF_Data_1.00e+00_3.00e+00.sdf


## Conclusion

This quickstart guide demonstrated the main features of KIKA:

1. Reading and parsing MCNP input files
2. Accessing material and perturbation data
3. Reading and analyzing MCTAL output files
4. Computing sensitivity data for specific nuclides
5. Creating and writing SDF files for uncertainty analysis

For more advanced usage, please refer to the documentation and additional examples.